## Priority Queue

In [1]:
class PriorityQueue():
    def __init__(self,item,cost):
        self.items = {cost:[item]}
        self.costs = {cost}
    
    def isEmpty(self):
        return self.items == {}
    
    def dequeue(self):
        least_cost = sorted(self.costs)[0]
        item = self.items[least_cost].pop(0)
        if len(self.items[least_cost]) == 0:
            self.costs.remove(least_cost)
            del self.items[least_cost]
        return item
    
    def enqueue(self,item,cost):
        if cost in self.costs:
            self.items[cost].append(item)    
        else:
            self.items[cost] = [item]
            self.costs.add(cost)
            
    def update_cost(self,item,old_cost,new_cost):
        for i in self.items[old_cost]:
            if i.board_config == item.board_config:
                self.items[old_cost].remove(i)
                break
        if len(self.items[old_cost]) == 0:
            self.costs.remove(old_cost)
            del self.items[old_cost]
        if new_cost in self.costs:
            self.items[new_cost].append(item)
        else:
            self.items[new_cost] = [item]
            self.costs.add(new_cost)

## 8-puzzle state

In [2]:
class State():
    right = {0,1,3,4,6,7}
    left = {1,2,4,5,7,8}
    up = {3,4,5,6,7,8}
    down = {0,1,2,3,4,5}
    
    def __init__(self, board_config, parent, move, depth):
        self.board_config = board_config #board configuration of the current state in a string
        self.board_config_list = list(map(int,board_config.split(','))) #board configuration of the current state in a list
        self.i = self.board_config_list.index(0) #index of empty space in board (index of 0 in this case)
        self.parent = parent #parent state (node) of the present state
        self.move = move #the move (Up,Down,Left,Right) made in parent state that results in the present state 
        self.depth = depth #depth of the node in the search tree
        
    def get_children(self):
        """returns the list of all possible states reachable from the current state,
        each child in the list is a State object"""
        children = []
        if self.i in State.up:
            new_board_config = self.board_config_list[:]
            new_board_config[self.i],new_board_config[self.i-3] = new_board_config[self.i-3],new_board_config[self.i]
            children.append(State(','.join(map(str,new_board_config)),self.board_config,'Up',self.depth+1))
        if self.i in State.down:
            new_board_config = self.board_config_list[:]
            new_board_config[self.i],new_board_config[self.i+3] = new_board_config[self.i+3],new_board_config[self.i]
            children.append(State(','.join(map(str,new_board_config)),self.board_config,'Down',self.depth+1))
        if self.i in State.left:
            new_board_config = self.board_config_list[:]
            new_board_config[self.i],new_board_config[self.i-1] = new_board_config[self.i-1],new_board_config[self.i]
            children.append(State(','.join(map(str,new_board_config)),self.board_config,'Left',self.depth+1))
        if self.i in State.right:
            new_board_config = self.board_config_list[:]
            new_board_config[self.i],new_board_config[self.i+1] = new_board_config[self.i+1],new_board_config[self.i]
            children.append(State(','.join(map(str,new_board_config)),self.board_config,'Right',self.depth+1))
        return children
    def __str__(self):
        return self.board_config

## UCS implementation

In [26]:
def ucs(initial_state, goal):
    frontier = PriorityQueue(initial_state, 0)
    frontier_config_dict = {initial_state.board_config: 0}
    explored = set()
    search_space = {}
    
    while not frontier.isEmpty():
        state = frontier.dequeue()
        explored.add(state.board_config)
        search_space[state.board_config] = state
        
        if state.board_config == goal:
            path = []
            current_state = state
            
            while not current_state.parent == None:
                path.insert(0, current_state.move)
                current_state = search_space[current_state.parent]
            
            return path, len(path)
        
        for child in state.get_children():
            if child.board_config not in frontier_config_dict.keys() and child.board_config not in explored:
                child_cost = child.depth
                frontier.enqueue(child, child_cost)
                frontier_config_dict[child.board_config] = child_cost
                
            elif child.board_config in frontier_config_dict.keys():
                new_cost = child.depth
                old_cost = frontier_config_dict[child.board_config]
                if new_cost < old_cost:
                    frontier.update_cost(child, old_cost, new_cost)
                    frontier_config_dict[child.board_config] = new_cost
    
    return "Failure"

In [33]:
# Driver Code
import time

start_time = time.time()
start = '5,3,0,7,1,2,8,4,6' 
goal = '0,1,2,3,4,5,6,7,8'
initial_state = State(start, None, None, 0)

print(ucs(initial_state, goal))
end_time = time.time()
print("time taken by ucs:",end_time - start_time) #print the time taken for execution

(['Down', 'Down', 'Left', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Up', 'Left', 'Up'], 24)
time taken by ucs: 2.466937780380249
